## Analysis of geographical information on the Repertoire

The Repertoir includes two lists of Jesuits residences in China.

One reports to 1644 [Planche: Carte des Chrétientés Chinoises de la fin des Ming (1644)](../sources/dehergne-locations-1644.cli)
and the second to 1701  [VII. Carte des résidences de Chine en l'année 1701](../sources/dehergne-locations-1701.cli).


In [49]:

from timelink.notebooks import TimelinkNotebook

tlnb = TimelinkNotebook()
tlnb.print_info(show_token=True)

Timelink version: 1.1.24
Project name: dehergne-locations
Project home: /Users/jrc/mhk-home/sources/dehergne-locations
Database type: sqlite
Database name: dehergne_locations
Kleio image: timelinkserver/kleio-server
Kleio server token: ordbYw3SWmt9RN4ybFwUM8eBLTDu0oOS
Kleio server URL: http://127.0.0.1:8088
Kleio server home: /Users/jrc/mhk-home/sources/dehergne-locations
Kleio server container: upbeat_kilby
Kleio version requested: latest
Kleio server version: 12.8.593 (2025-03-16 21:55:53)
SQLite directory: /Users/jrc/mhk-home/sources/dehergne-locations/database/sqlite
Database version: 6ccf1ef385a6
Call print_info(show_password=True) to show the Postgres password
TimelinkNotebook(project_name=dehergne-locations, project_home=/Users/jrc/mhk-home/sources/dehergne-locations, db_type=sqlite, db_name=dehergne_locations, kleio_image=timelinkserver/kleio-server, kleio_version=latest, postgres_image=postgres, postgres_version=latest)


In [50]:
import re

def get_wikidata_id(geo_entity, if_missing=''):
    """ Check the obs field for wikidata links

    Returns a tuple of the cleaned comment and the wikidata id"""
    extra_info = geo_entity.extra_info
    name_comment = extra_info.get('name', {}).get('comment','')
    name_original = extra_info.get('name', {}).get('original','')

    pattern = r'@wikidata\:\s*(Q[0-9]*)'
    wikidata_in_comment = re.findall(pattern, name_comment)
    comment_without_wikidata = re.sub(pattern, '', name_comment)
    # Sometimes the wikidata id is in the original name
    wikidata_in_original = re.findall(pattern, name_original)
    original_without_wikidata = re.sub(pattern, '', name_original)
    return comment_without_wikidata + original_without_wikidata, wikidata_in_comment[0] if wikidata_in_comment else wikidata_in_original[0] if wikidata_in_original else if_missing



## List of residences in 1644

In [3]:
import re
import pandas as pd
from sqlalchemy import select

geo1, geo2, geo3 = tlnb.db.get_model(['geo1','geo2','geo3'])
stmt = select(geo1).where(geo1.inside == 'deh-chre-1644')

place_list = []

with tlnb.db.session() as session:
    result = session.execute(stmt).fetchall()
    for province, in result:
        comment, wikidata = get_wikidata_id(province, if_missing='No wikidata')
        print(province.name, wikidata, comment)
        place_list.append({'province': province.name,'fou':'',  'name': province.name, 'wikidata': wikidata, 'comment': comment})
        fous = session.execute(select(geo2).where(geo2.inside == province.id)).fetchall()
        for fou, in fous:
            comment, wikidata = get_wikidata_id(fou, if_missing='No wikidata')
            print(' ', fou.name,  wikidata, comment)
            place_list.append({'province': province.name, 'fou':fou.name,  'name': fou.name, 'wikidata': wikidata, 'comment': comment})
            geo3s = session.execute(select(geo3).where(geo3.inside == fou.id)).fetchall()
            for tcheou_hien, in geo3s:
                comment, wikidata = get_wikidata_id(tcheou_hien, if_missing='No wikidata')
                print('   ', tcheou_hien.name, wikidata,comment,  )
                place_list.append({'province': province.name, 'fou':fou.name,  'name': tcheou_hien.name, 'wikidata': wikidata, 'comment': comment})


Chekiang Q16967 Tche-kiang, hoje:Zhejiang, 浙江,  @dehergne:396
  Hangchou Q4970 Hang-tcheou, hoje: Hangzhou, 杭州, 
    Fuyang Q1011103 Fou-yang, hoje:Fuyang, 富阳, 
    Jenho No wikidata Jen-houo, hoje: Renhe, 仁和县, Historical county name. coordinate: 30.448897N, 120.307504E
  Chüchow Q58235 K'iu-tcheou, hoje:Quzhou, 衢州, , in the Chinese translation it is recognized as “遂州”, which is wrong, both phonetically and geographically. In Dehergne(1957), it is noted as "衢州".
  Huchow Q42664 Hou-tcheou, hoje: Huzhou, 湖州, 
    Tehtsing Q1191987 "Tehtsing du Huchow, Té-ts'ing;Teching, hoje: Deqing, 德清, "
  Kashing Q58178 Kia-hing, hoje:Jiaxing, 嘉兴, 
    Kashan Q1361347 Kia-chan, hoje: Jiashan, 嘉善, Kaosham
    Tangsi Q10931032 "T'ang-k'i Tangchi", hoje:Tangqi, 塘栖 , in the Chinese translation it is recognized as “塘拪”
    Tsungteh Q10270889 Tch'ong-té,hoje: Chongde, 崇德县 , Historical county name, located in the present Chongfu 崇福镇Tsungteh (Shihmen)
    Tungsiang Q1204548 T'ong-hiang, hoje:Tongxiang, 桐乡, 


In [20]:
pd.set_option('display.max_rows', 300)
# create a dataframe from the list
places_1644_df = pd.DataFrame(place_list)
places_1644_df

,province,fou,name,wikidata,comment
0,Chekiang,,Chekiang,Q16967,Zhejiang， CHEKIANG (actuel ZHEJIANG)
1,Chekiang,Hangchou,Hangchou,Q4970,"Hangzhou,"
2,Chekiang,Hangchou,Haining,Q286266,海宁
3,Chekiang,Kashing,Kashing,Q58178,"Jiaxing,"
4,Chekiang,Kinhwa,Kinhwa,Q58210,"Jinhua,"
5,Chekiang,Kinhwa,Lanchi,Q1023793,"Lanxi,"
6,Chekiang,Ningpo,Ningpo,Q42780,"Ningbo,"
7,Chekiang,Ningpo,Yenchow,Q1334217,"Yinzhou, 鄞州"
8,Fukien,,Fukien,Q41705,"Fujian,"
9,Fukien,Foochow,Foochow,Q68481,"Fuzhou,"


In [51]:
places_1644_df.to_excel('../inferences/places-1644.xlsx', index=False)

## 1701

In [52]:
stmt = select(geo1).where(geo1.inside == 'deh-chre-1701')

place_list = []

with tlnb.db.session() as session:
    result = session.execute(stmt).fetchall()
    for province, in result:
        comment, wikidata = get_wikidata_id(province, if_missing='No wikidata')
        print(province.name, wikidata, comment)
        place_list.append({'province': province.name,'fou':'',  'name': province.name, 'wikidata': wikidata, 'comment': comment})
        fous = session.execute(select(geo2).where(geo2.inside == province.id)).fetchall()
        for fou, in fous:
            comment, wikidata = get_wikidata_id(fou, if_missing='No wikidata')
            print(' ', fou.name,  wikidata, comment)
            place_list.append({'province': province.name, 'fou':fou.name,  'name': fou.name, 'wikidata': wikidata, 'comment': comment})
            geo3s = session.execute(select(geo3).where(geo3.inside == fou.id)).fetchall()
            for tcheou_hien, in geo3s:
                comment, wikidata = get_wikidata_id(tcheou_hien, if_missing='No wikidata')
                print('   ', tcheou_hien.name, wikidata,comment,  )
                place_list.append({'province': province.name, 'fou':fou.name,  'name': tcheou_hien.name, 'wikidata': wikidata, 'comment': comment})


Chekiang Q16967 Zhejiang 浙江， CHEKIANG (actuel ZHEJIANG)
  Hangchou Q4970 Hangzhou 杭州, 
    Haining Q286266 海宁 
  Kashing Q58178 Jiaxing 嘉兴, 
  Kinhwa Q58210 Jinhua 金华, 
    Lanchi Q1023793 Lanxi 兰溪, 
  Ningpo Q42780 Ningbo 宁波, 
    Yenchow Q1334217 Yinzhou 鄞州, 
Fukien Q41705 Fujian 福建, 
  Foochow Q68481 Fuzhou 福州, 
    Lienkong Q204827 Lianjiang 连江, 
  Changchow Q68814 Zhangzhou 漳州, 
  Chüanchow Q68695 Quanzhou 泉州, 
    Amoy Q68744 Xiamen 厦门, 
  Funing Q241877 (alors tcheou indépendant)Funing, , hoje Xiapu 霞浦
    Fuan Q1374581 Fu'an 福安, 
  Hinghwa Q17498990 Xinghua 兴化, , 兴化县，the wikidata code of it dose not contain coordinate information, which can be substituted by Q68579，the present-day Putian 莆田
  Kienning Q11065314 Kienning fou, Jianning 建寧,  the wikidata code of it does not contain coordinate information, which can be substituted by Kien-yang today Jianyang Q639862 see 1644
    Pucheng Q1338032 浦城 
  Shaowu Q1025451 Shaowu 邵武, 
    Kienning hien Q781559 Jianning Xian 建宁, 
  Tingch

In [53]:
pd.set_option('display.max_rows', 300)
# create a dataframe from the list
places_1701_df = pd.DataFrame(place_list)
places_1701_df

,province,fou,name,wikidata,comment
0,Chekiang,,Chekiang,Q16967,Zhejiang 浙江， CHEKIANG (actuel ZHEJIANG)
1,Chekiang,Hangchou,Hangchou,Q4970,"Hangzhou 杭州,"
2,Chekiang,Hangchou,Haining,Q286266,海宁
3,Chekiang,Kashing,Kashing,Q58178,"Jiaxing 嘉兴,"
4,Chekiang,Kinhwa,Kinhwa,Q58210,"Jinhua 金华,"
5,Chekiang,Kinhwa,Lanchi,Q1023793,"Lanxi 兰溪,"
6,Chekiang,Ningpo,Ningpo,Q42780,"Ningbo 宁波,"
7,Chekiang,Ningpo,Yenchow,Q1334217,"Yinzhou 鄞州,"
8,Fukien,,Fukien,Q41705,"Fujian 福建,"
9,Fukien,Foochow,Foochow,Q68481,"Fuzhou 福州,"


In [55]:
places_1701_df.to_excel('../inferences/places-1701.xlsx', index=False)

## Join the two lists

In [54]:
places_1644_df['year'] = 1644
places_1701_df['year'] = 1701
places_df = pd.concat([places_1644_df, places_1701_df], ignore_index=True)
places_df.sort_values(by=['province', 'fou', 'name','year'], inplace=True)
places_df[['year','province', 'fou', 'name', 'wikidata', 'comment', ]].to_excel('../inferences/places-1644-1701.xlsx', index=False)
places_df[['year','province', 'fou', 'name', 'wikidata', 'comment', ]]

,year,province,fou,name,wikidata,comment
0,1644,Chekiang,,Chekiang,Q16967,Zhejiang， CHEKIANG (actuel ZHEJIANG)
145,1701,Chekiang,,Chekiang,Q16967,Zhejiang 浙江， CHEKIANG (actuel ZHEJIANG)
2,1644,Chekiang,Hangchou,Haining,Q286266,海宁
147,1701,Chekiang,Hangchou,Haining,Q286266,海宁
1,1644,Chekiang,Hangchou,Hangchou,Q4970,"Hangzhou,"
146,1701,Chekiang,Hangchou,Hangchou,Q4970,"Hangzhou 杭州,"
3,1644,Chekiang,Kashing,Kashing,Q58178,"Jiaxing,"
148,1701,Chekiang,Kashing,Kashing,Q58178,"Jiaxing 嘉兴,"
4,1644,Chekiang,Kinhwa,Kinhwa,Q58210,"Jinhua,"
149,1701,Chekiang,Kinhwa,Kinhwa,Q58210,"Jinhua 金华,"


## List all the occurences of a place in the biographies

In [56]:
list(tlnb.db.get_view('nattributes').columns)

[Column('id', String(), table=<nattributes>, primary_key=True, nullable=False),
 Column('name', String(), table=<nattributes>),
 Column('sex', String(length=1), table=<nattributes>),
 Column('attr_id', String(), table=<nattributes>, primary_key=True, nullable=False),
 Column('the_type', String(), table=<nattributes>),
 Column('the_value', String(), table=<nattributes>),
 Column('the_date', String(), table=<nattributes>),
 Column('pobs', String(), table=<nattributes>),
 Column('aobs', String(), table=<nattributes>)]

In [58]:
import re
import pandas as pd
from sqlalchemy import select

from timelink.kleio.utilities import convert_timelink_date, format_timelink_date

geo1, geo2, geo3 = tlnb.db.get_model(['geo1','geo2','geo3'])
nattributes = tlnb.db.get_view('nattributes')
stmt = select(geo1).where(geo1.the_type == "geo1").order_by(geo1.name,geo1.the_source)

place_list = []

def show_nattributes_for_wikidata(wikidata, ident='', session=None):
    """ Show the nattribute for a given wikidata id
    """
    wikidata_uri = 'https://www.wikidata.org/wiki/' + wikidata
    stmt = select(nattributes).where(nattributes.c.the_value == wikidata_uri).order_by(nattributes.c.the_date)
    results = session.execute(stmt).fetchall()
    for result in results:
        the_date = convert_timelink_date(result.the_date)
        fdate = format_timelink_date(result.the_date)
        print(f"{ident}{fdate} {result.name} {result.id} {result.pobs} ({result.the_date})")

with tlnb.db.session() as session:
    result = session.execute(stmt).fetchall()
    for province, in result:
        print()
        comment, wikidata = get_wikidata_id(province, if_missing='No wikidata')
        print(province.name, wikidata, comment)
        place_list.append({'province': province.name,'fou':'',  'name': province.name, 'wikidata': wikidata, 'comment': comment})

        show_nattributes_for_wikidata(wikidata, '  ', session=session)
        # search all the entities that have this value in one of the attributes
        fous = session.execute(select(geo2).where(geo2.inside == province.id)).fetchall()
        for fou, in fous:
            print()
            comment, wikidata = get_wikidata_id(fou, if_missing='No wikidata')
            print(' ', fou.name,  wikidata, comment)
            place_list.append({'province': province.name, 'fou':fou.name,  'name': fou.name, 'wikidata': wikidata, 'comment': comment})
            show_nattributes_for_wikidata(wikidata, '   ', session=session)
            geo3s = session.execute(select(geo3).where(geo3.inside == fou.id)).fetchall()
            for tcheou_hien, in geo3s:
                print()
                comment, wikidata = get_wikidata_id(tcheou_hien, if_missing='No wikidata')
                print('   ', tcheou_hien.name, wikidata,comment,  )
                place_list.append({'province': province.name, 'fou':fou.name,  'name': tcheou_hien.name, 'wikidata': wikidata, 'comment': comment})
                show_nattributes_for_wikidata(wikidata, '     ', session=session)



Anhwei Q40956 Anhui, Ngon-hoei, hoje:Anhui, 安徽, 

  Chuchow Q114045 Chuchow, hoje:Chuzhou, 滁州, 

  Hweichow Q4358404 Hoei-tcheou, hoje:Huizhou, 徽州, 

    Wuyan hien Q1357710 Ou-yuen, hoje:Wuyuan Xian, 婺源县, , Wuyuan historically was in Anhui province, but now it is in the present-day Jiangxi province

    Tungmen No wikidata 

Chekiang Q16967 Tche-kiang, hoje:Zhejiang, 浙江,  @dehergne:396
   Jan Baptista Chrzciciel Bakowski deh-jan-baptista-chrzciciel-bakowski  (0)
   Antonio Faglia deh-antonio-faglia  (0)
   Álvaro Semedo deh-alvaro-semedo  (0)
  1617-10-02 Gaspar Ferreira deh-gaspar-ferreira  (16171002)
  1639 Agostino Tudeschini deh-agostino-tudeschini  (16390000)
  1661 Philippe Couplet deh-philippe-couplet  (16610000)
  1688 Paul Banhes Wan deh-paul-banhes-wan  (16880000)
  1706 José Pereira deh-jose-pereira  (17060000)
  1710 Joseph Marie Anne de Moyriac Mailla deh-joseph-marie-anne-de-moyriac-mailla  (17100000)
  1713 Romain Hinderer deh-romain-hinderer  (17130000)
  1714 Romain 

/Users/jrc/mhk-home/sources/dehergne-locations/.venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3577: UserWarning: Invalid date: 16171240 -> unconverted data remains: 0
  exec(code_obj, self.user_global_ns, self.user_ns)
/var/folders/x7/qzytnd752559g0w475rfwkx00000gn/T/ipykernel_91638/352083337.py:40: UserWarning: Invalid date: 16171240 -> unconverted data remains: 0
  show_nattributes_for_wikidata(wikidata, '   ', session=session)



Kwangsi Q15176 Guangxi, Koang-si, hoje: Guangxi, 广西, 
   Jan Baptista Chrzciciel Bakowski deh-jan-baptista-chrzciciel-bakowski  (0)
   Domingos de Brito deh-domingos-de-brito  (0)
  1650 Michael-Pierre Boym deh-michael-pierre-boym  (16500000)
  1650 Yong-Li deh-michael-pierre-boym-ref1  (16500000)
  1652-12-12 Andreas Wolfgang Koffler deh-andreas-wolfgang-koffler  (16521212)

  Kweilin Q189633 Koei-lin, hoje: Guilin, 桂林, 
    Jean-Simon Bayard deh-jean-simon-bayard  (0)
   1646 Andreas Wolfgang Koffler deh-andreas-wolfgang-koffler  (16460000)
   1693-01-09 Diogo Vidal deh-diogo-vidal  (16930109)

  Wuchow Q571652 Ou-tcheou, hoje: Wuzhou, 梧州, 

Kwangsi Q15176 Guangxi 广西, 
   Jan Baptista Chrzciciel Bakowski deh-jan-baptista-chrzciciel-bakowski  (0)
   Domingos de Brito deh-domingos-de-brito  (0)
  1650 Michael-Pierre Boym deh-michael-pierre-boym  (16500000)
  1650 Yong-Li deh-michael-pierre-boym-ref1  (16500000)
  1652-12-12 Andreas Wolfgang Koffler deh-andreas-wolfgang-koffler  (16521

/Users/jrc/mhk-home/sources/dehergne-locations/.venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3577: UserWarning: Invalid date: 161910715 -> unconverted data remains: 15
  exec(code_obj, self.user_global_ns, self.user_ns)
/var/folders/x7/qzytnd752559g0w475rfwkx00000gn/T/ipykernel_91638/352083337.py:40: UserWarning: Invalid date: 161910715 -> unconverted data remains: 15
  show_nattributes_for_wikidata(wikidata, '   ', session=session)
/Users/jrc/mhk-home/sources/dehergne-locations/.venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3577: UserWarning: Invalid date: 165990000 -> time data '165990000' does not match format '%Y%m%d'
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/jrc/mhk-home/sources/dehergne-locations/.venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3577: UserWarning: Invalid date: 170902022 -> unconverted data remains: 2
  exec(code_obj, self.user_global_ns, self.user_ns)
/var/folders/x7/qzytnd752559g0w475

   1780-12-24 Jacques-François-Dieudonné d'Ollières deh-jacques-francois-dieudonne-dollieres  (17801224)
   1781-01-22 Jean-Paul Louis Collas deh-jean-paul-louis-collas  (17810122)
   1781-05-18 Mateus de Sequeira deh-mateus-de-sequeira  (17810518)
   1781-05-18 Almeida deh-mateus-de-sequeira-ref1 não especifica qual deles (17810518)
   1781-05-22 Félix da Rocha deh-felix-da-rocha  (17810522)
   1781-06-25 Mateus de Sequeira deh-mateus-de-sequeira  (17810625)
   1781-06-25 Almeida deh-mateus-de-sequeira-ref1 não especifica qual deles (17810625)
   1784 Seng-Koun-Yi deh-jean-baptiste-joseph-de-grammont-ref1  (17840000)
   1785 Jean Simonelli Ngai deh-jean-simonelli-ngai  (17850000)
   1785-01-22 João de Seixas deh-joao-de-seixas  (17850122)
   1785-02-11 Jean Simonelli Ngai deh-jean-simonelli-ngai  (17850211)
   1787-05-27 Jean-Matthieu Tournu Ventavon deh-jean-matthieu-tournu-ventavon teve três tios e um tio-avô jesuítas (17870527)
   1788-07-10 José de Espinha deh-jose-de-espinha  (17